This is just a demostration on how I got the regional averaged values

In [1]:
import os
import numpy as np
import pandas as pd
from nibabel.freesurfer.io import read_morph_data, read_annot, read_geometry

In [2]:
masterfile_path = "/work/fsammarco/datasets/IXI/subjects.csv" # This is different from the one you have availble in the lessons...
masterfile = pd.read_csv(masterfile_path)
masterfile.head(5)

,subject_id,age,sex,scanner,site,diagnosis,dataset_name,subject_key,session,run,registered_mni
0,IXI002,35.80,Female,Philips-1.5T,Guy’s-Hospital,Healthy,IXI,IXI002_IXI,1,1,sub-IXI002/ses-1/run-1/anat/sub-IXI002_acq-Phi...
1,IXI012,38.78,Male,Philips-3.0T,Hammersmith-Hospital,Healthy,IXI,IXI012_IXI,1,1,sub-IXI012/ses-1/run-1/anat/sub-IXI012_acq-Phi...
2,IXI013,46.71,Male,Philips-3.0T,Hammersmith-Hospital,Healthy,IXI,IXI013_IXI,1,1,sub-IXI013/ses-1/run-1/anat/sub-IXI013_acq-Phi...
3,IXI014,34.24,Female,Philips-3.0T,Hammersmith-Hospital,Healthy,IXI,IXI014_IXI,1,1,sub-IXI014/ses-1/run-1/anat/sub-IXI014_acq-Phi...
4,IXI015,24.28,Male,Philips-3.0T,Hammersmith-Hospital,Healthy,IXI,IXI015_IXI,1,1,sub-IXI015/ses-1/run-1/anat/sub-IXI015_acq-Phi...


In [6]:
def get_paths(hemi, row):
    base = "/var/datasets/IXI/freesurfer8/preproc_subjects"
    row[f"{hemi}_thickness"] = os.path.join(base, f'sub-{row["subject_id"]}', "surf", f"{hemi}.thickness")
    row[f"{hemi}_pial"] = os.path.join(base, f'sub-{row["subject_id"]}', "surf", f"{hemi}.pial")
    row[f"{hemi}_annot"] = os.path.join(base, f'sub-{row["subject_id"]}', "label", f"{hemi}.aparc.DKTatlas.annot")
    return row

masterfile = masterfile.apply(lambda x: get_paths("lh", x), axis=1)
masterfile = masterfile.apply(lambda x: get_paths("rh", x), axis=1)

In [17]:
"""
Compute area-weighted and simple mean thickness per region for one subject and both hemispheres.
annotation: one of "aparc", "aparc.DKTatlas", "aparc.a2009s" (matching the .annot filename)
Returns a DataFrame with columns: subject, hemi, region, mean_thickness_weighted, mean_thickness_simple
"""
results = []
for _, row in masterfile.iterrows():
    for hemi in ("lh", "rh"):
        # subject id
        subject_id = row["subject_id"]

        # paths
        thickness_path = row[f"{hemi}_thickness"]
        print(thickness_path)
        annot_path = row[f"{hemi}_annot"]
        print(annot_path)
        surface_path = row[f"{hemi}_pial"]
        print(surface_path)
        print("Processing...")
        
        # sanity check
        if not (os.path.isfile(thickness_path) and os.path.isfile(annot_path) and os.path.isfile(surface_path)):
            raise FileNotFoundError(f"Missing file for subject={subject_id}, hemi={hemi}")
        
        # load data
        thickness = read_morph_data(thickness_path)  # per-vertex
        labels, ctab, names = read_annot(annot_path)
        coords, faces = read_geometry(surface_path)
        
        # compute face areas
        v0 = coords[faces[:, 0], :]
        v1 = coords[faces[:, 1], :]
        v2 = coords[faces[:, 2], :]
        face_areas = 0.5 * np.linalg.norm(np.cross(v1 - v0, v2 - v0), axis=1)
        
        # distribute to vertices (1/3 each adjacent face)
        vertex_area = np.zeros(coords.shape[0])
        for vi in range(3):
            np.add.at(vertex_area, faces[:, vi], face_areas / 3.0)
        
        # region-wise means
        unique_labels = np.unique(labels)
        for lab in unique_labels:
            mask = labels == lab
            if not np.any(mask):
                continue
            areas = vertex_area[mask]

            total_area = float(np.sum(areas))
            n_vertices = int(mask.sum())

            t_vals = thickness[mask]
            mean_thick_weighted = np.sum(t_vals * areas) / total_area
            mean_thick_simple = np.mean(t_vals)

            if int(lab) == -1:
                region_name = "unknown"
            else:
                region_name = names[int(lab)].decode("utf-8")
                
            results.append({
                "subject_id": subject_id,
                "hemi": hemi,
                "region": region_name,
                "mean_thickness_weighted": mean_thick_weighted,
                "mean_thickness_simple": mean_thick_simple,
                "n_vertices": n_vertices,
                "total_area": total_area
            })
thickness_df = pd.DataFrame(results)

/var/datasets/IXI/freesurfer8/preproc_subjects/sub-IXI002/surf/lh.thickness
/var/datasets/IXI/freesurfer8/preproc_subjects/sub-IXI002/label/lh.aparc.DKTatlas.annot
/var/datasets/IXI/freesurfer8/preproc_subjects/sub-IXI002/surf/lh.pial
Processing...
/var/datasets/IXI/freesurfer8/preproc_subjects/sub-IXI002/surf/rh.thickness
/var/datasets/IXI/freesurfer8/preproc_subjects/sub-IXI002/label/rh.aparc.DKTatlas.annot
/var/datasets/IXI/freesurfer8/preproc_subjects/sub-IXI002/surf/rh.pial
Processing...
/var/datasets/IXI/freesurfer8/preproc_subjects/sub-IXI012/surf/lh.thickness
/var/datasets/IXI/freesurfer8/preproc_subjects/sub-IXI012/label/lh.aparc.DKTatlas.annot
/var/datasets/IXI/freesurfer8/preproc_subjects/sub-IXI012/surf/lh.pial
Processing...
/var/datasets/IXI/freesurfer8/preproc_subjects/sub-IXI012/surf/rh.thickness
/var/datasets/IXI/freesurfer8/preproc_subjects/sub-IXI012/label/rh.aparc.DKTatlas.annot
/var/datasets/IXI/freesurfer8/preproc_subjects/sub-IXI012/surf/rh.pial
Processing...
/var

In [18]:
thickness_df 

,subject_id,hemi,region,mean_thickness_weighted,mean_thickness_simple,n_vertices,total_area
0,IXI002,lh,unknown,0.000369,0.000338,8853,5502.432653
1,IXI002,lh,caudalanteriorcingulate,2.767575,2.609781,1437,1083.926104
2,IXI002,lh,caudalmiddlefrontal,2.828643,2.690581,3830,2769.405243
3,IXI002,lh,cuneus,2.019888,2.020947,2739,2037.500121
4,IXI002,lh,entorhinal,2.712761,2.636525,708,687.427367
...,...,...,...,...,...,...,...
37063,IXI662,rh,superiorparietal,2.120859,2.023058,6653,5314.046982
37064,IXI662,rh,superiortemporal,2.965944,2.796967,8804,7119.010455
37065,IXI662,rh,supramarginal,2.508402,2.370133,5606,4399.206890
37066,IXI662,rh,transversetemporal,2.354042,2.350365,628,446.668808


In [19]:
thickness_df.to_csv("../../data/IXI/thickness.csv", index=False)